In [20]:
import pandas as pd
import numpy as np
from collections import Counter

## Decision Tree

In [21]:
class Node():
    #Initializes a new Node instance with the specified attributes.
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        """
        Args:
        - feature_index: Index of the feature used for splitting the dataset.
        - threshold: Threshold value for the feature used in the splitting.
        - left: Reference to the left subtree node.
        - right: Reference to the right subtree node.
        - info_gain: Information gain achieved after the node split.
        - value: Value assigned to the node if it is a leaf node. Default is None.
        """
        
        self.feature_index = feature_index #Which feature the tree was divided with
        self.threshold = threshold #Which threshold the tree was divided with
        self.left = left #Left node we are pointing to
        self.right = right #Right node we are pointing to
        self.info_gain = info_gain
        self.value = value #Value of the Node if its a Leaf Node

In [22]:
class DecisionTree():
    def __init__(self, min_samples_split=2, max_depth=2, n_features=None):
        #Stopping criteria
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
        self.n_features = n_features
        
        self.root = None #initialize the root of the tree
        
    
    def build_tree(self, dataset, curr_depth=0):
        """
        Recursively builds a decision tree based on the given dataset.
        Select the best feature to split on and create left and right subtrees until stopping conditions are met.

        Args:
        - dataset: Input dataset containing features and labels.
        - curr_depth: Current depth of the tree.

        Returns:
        - Node: The root node of the constructed decision tree.
        """

        X, Y = dataset[:, :-1], dataset[:, -1]
        num_samples, num_features = np.shape(X)

        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:

            feature_indices = np.random.choice(num_features, self.n_features, replace=False)
            best_split = self.get_best_split(dataset, num_samples, num_features)

            if best_split and best_split["info_gain"] > 0:
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth + 1)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth + 1)

                return Node(best_split["feature_index"], best_split["threshold"],
                            left_subtree, right_subtree, best_split["info_gain"])

        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)
       
    
    def get_best_split(self, dataset, num_samples, num_features):
        """
        Finds the best split for a decision tree node based on information gain and Gini impurity.

        Args:
        - dataset: Input dataset containing features and labels.
        - num_samples: Number of samples in the dataset.
        - num_features: Number of features in the dataset.

        Returns:
        - Dictionary containing information about the best split.
        """
        
        best_split = {}
        max_info_gain = -float("inf")
        
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_threshold = np.unique(feature_values)
            
            for threshold in possible_threshold:
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    y, left_y, right_y = dataset[:,-1], dataset_left[:,-1], dataset_right[:,-1]
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    
                    if curr_info_gain > max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
        return best_split
      
    
    def split(self, dataset, feature_index, threshold):
        dataset_left = dataset[dataset[:, feature_index] <= threshold]
        dataset_right = dataset[dataset[:, feature_index] > threshold]
        return dataset_left, dataset_right
   
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        """
        Calculates the information gain based on the specified splitting criterion.

        Args:
        - parent: Labels of the parent dataset.
        - l_child: Labels of the left child dataset after the split.
        - r_child: Labels of the right child dataset after the split.
        - mode: The splitting criterion to use, either 'entropy' or 'gini'.

        Returns:
        - Information gain achieved by the split.
        """       
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode == "gini":
            gain = self.gini_index(parent) - (weight_l * self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l * self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
      
    
    def entropy(self, y):
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy = entropy - p_cls * np.log2(p_cls)
        return entropy
   
    
    def gini_index(self, y):
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini = gini + p_cls ** 2
        return 1 - gini
    
    
    def calculate_leaf_value(self, Y):
        Y = list(Y)
        return max(Y, key=Y.count) #finds the class label that occurs most frequently in the list Y
    
    
    def print_tree(self, tree=None, indent=" "):
        """
        Prints the structure of the decision tree.

        Args:
        - tree: The root node of the tree/subtree.
        - indent: The indentation used for structuring.
        """

        if not tree:
            tree = self.root
        if tree.value is not None:
            print(tree.value)
        else:            
            print(f"X_{tree.feature_index} <= {tree.threshold} ? {tree.info_gain}")
            print(f"{indent}left:", end="")
            self.print_tree(tree.left, indent + indent)
            print(f"{indent}right:", end="")
            self.print_tree(tree.right, indent + indent)
    
    
    def fit(self, X, Y):
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
        
    
    def predict(self, X):
        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions
    
    
    def make_prediction(self, x, tree):
        if tree.value!=None:
            return tree.value
        
        feature_val = x[tree.feature_index]
        if feature_val <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

In [23]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

data = datasets.load_breast_cancer()
X, Y = data.data, data.target.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

classifier = DecisionTree(min_samples_split=3, max_depth=3)
classifier.fit(X_train,y_train)
classifier.print_tree()

X_27 <= 0.1423 ? 0.3202719907491167
 left:X_23 <= 947.9 ? 0.06459284129733107
  left:X_22 <= 107.4 ? 0.010885658029099604
    left:X_10 <= 0.8811 ? 0.0071878415580914345
        left:1.0
        right:0.0
    right:X_0 <= 13.96 ? 0.2496768236380425
        left:0.0
        right:1.0
  right:X_8 <= 0.1495 ? 0.23111111111111104
    left:1.0
    right:0.0
 right:X_23 <= 719.8 ? 0.05053412277834146
  left:X_4 <= 0.1078 ? 0.48979591836734704
    left:1.0
    right:0.0
  right:X_10 <= 0.163 ? 0.013970720130255621
    left:1.0
    right:X_26 <= 0.1804 ? 0.014279647604153158
        left:1.0
        right:0.0


In [24]:
Y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score
print("Accuracy is:")
accuracy_score(y_test, Y_pred)

Accuracy is:


0.956140350877193

## Random Forest 

In [28]:
class RandomForest:
    def __init__(self, n_trees=5, max_depth=5, min_samples_split=2, n_feature=None):
        self.n_trees = n_trees
        self.max_depth=max_depth
        self.min_samples_split=min_samples_split
        self.n_features=n_feature
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(max_depth=self.max_depth, min_samples_split=self.min_samples_split, n_features=self.n_features)
            X_sample, y_sample = self.bootstrap_samples(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    
    def bootstrap_samples(self, X, y):
        #Bootstrap samples from the training data
        
        n_samples = X.shape[0]
        random_idxs = np.random.choice(n_samples, n_samples, replace=True)
        return X[random_idxs], y[random_idxs] #returns bootstrap features and labels

    def most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common

    
    def predict(self, X):
        tree_predictions = [tree.predict(X) for tree in self.trees]
        transposed_predictions = np.swapaxes(tree_predictions, 0, 1) #Transpose the predictions
        final_predictions = [self.most_common_label(pred) for pred in transposed_predictions]
        return np.array(final_predictions)

In [29]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

data = datasets.load_breast_cancer()
X, Y = data.data, data.target.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

classifier = RandomForest()
classifier.fit(X_train,y_train)
predictions = classifier.predict(X_test)

In [30]:
Y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score
print("Accuracy is:")
accuracy_score(y_test, Y_pred)

Accuracy is:


0.9298245614035088